<a href="https://colab.research.google.com/github/Space9999/braintumorpredictor/blob/main/Brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, glob, keras
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import tensorflow as tf
import random
import pandas as pd
import cv2
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip Healthy.zip

unzip:  cannot find or open Healthy.zip, Healthy.zip.zip or Healthy.zip.ZIP.


In [ ]:
cd gdrive/MyDrive/Colab\ Notebooks/Images


/content/gdrive/MyDrive/Colab Notebooks/Images


In [ ]:
!unzip Healthy.zip

Archive:  Healthy.zip
   creating: Healthy/
  inflating: Healthy/ct_healthy (1).jpg  
  inflating: Healthy/ct_healthy (1).png  
  inflating: Healthy/ct_healthy (10).jpg  
  inflating: Healthy/ct_healthy (10).png  
  inflating: Healthy/ct_healthy (100).jpg  
  inflating: Healthy/ct_healthy (100).png  
  inflating: Healthy/ct_healthy (1000).jpg  
  inflating: Healthy/ct_healthy (1001).jpg  
  inflating: Healthy/ct_healthy (1002).jpg  
  inflating: Healthy/ct_healthy (1003).jpg  
  inflating: Healthy/ct_healthy (1004).jpg  
  inflating: Healthy/ct_healthy (1005).jpg  
  inflating: Healthy/ct_healthy (1006).jpg  
  inflating: Healthy/ct_healthy (1007).jpg  
  inflating: Healthy/ct_healthy (1008).jpg  
  inflating: Healthy/ct_healthy (1009).jpg  
  inflating: Healthy/ct_healthy (101).jpg  
  inflating: Healthy/ct_healthy (101).png  
  inflating: Healthy/ct_healthy (1010).jpg  
  inflating: Healthy/ct_healthy (1011).jpg  
  inflating: Healthy/ct_healthy (1012).jpg  
  inflating: Healthy/ct_h

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
# Open the image.
img = cv2.imread("Healthy/ct_healthy (999).jpg")
cv2_imshow(img)

AttributeError: 'NoneType' object has no attribute 'clip'

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Images

/content/drive/MyDrive/Colab Notebooks/Images


In [ ]:
!unzip archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (10).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (100).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1000).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1001).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1002).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1003).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1004).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1005).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1006).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1007).jpg  
  inflating: Dataset/Brain Tumor MRI images/Healthy/mri_healthy (1008).jpg  
  inflating: Dataset/Brain Tumo

In [1]:
import os, glob
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import tensorflow as tf
import random
import pandas as pd
import cv2
sns.set()

In [2]:
ctHealthyPath = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor CT scan Images", "Healthy", "*.jpg"))
ctTumorPath  = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor CT scan Images", "Tumor", "*.jpg"))
mriHealthyPath = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor MRI images", "Healthy", "*.jpg"))
mriTumorPath  = glob.glob(os.path.join("/content/drive/MyDrive/Colab Notebooks/Images/Dataset/Brain Tumor MRI images", "Tumor", "*.jpg"))
data = []

for path in ctHealthyPath:
    data.append((path, 0, "CT"))
for path in ctTumorPath:
    data.append((path, 1, "CT"))

for path in mriHealthyPath:
    data.append((path, 0, "MRI"))
for path in mriTumorPath:
    data.append((path, 1, "MRI"))

print("Number of Data entries:")
print("CT Healthy: ", len(ctHealthyPath))
print("CT Tumor: ", len(ctTumorPath))
print("MRI Healthy: ", len(mriHealthyPath))
print("MRI Tumor: ", len(mriTumorPath))
print("Total images in Data:", len(data))

Number of Data entries:
CT Healthy:  1716
CT Tumor:  2157
MRI Healthy:  1997
MRI Tumor:  2984
Total images in Data: 8854


In [3]:
labels = [i[1] for i in data]
train_paths, val_paths, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)
val_datagen = ImageDataGenerator(rescale=1.0/255)

In [5]:
def generate_images(image_paths, labels, batch_size=32, target_size=(224, 224)):
    # Ensure the generator always yields something, even if there are no images
    if not image_paths:  # If image_paths is empty
        yield np.empty((0, *target_size, 3)), np.empty((0,))  # yield empty arrays
    else:
        while True:
            for i in range(0, len(image_paths), batch_size):
                batch_paths = image_paths[i:i + batch_size]
                batch_labels = labels[i:i + batch_size]

                images = []
                for img_data in batch_paths:
                    img_path = img_data[0]
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
                    img = tf.keras.preprocessing.image.img_to_array(img)
                    images.append(img)

                images = np.array(images)
                yield images, np.array(batch_labels)
train_generator = generate_images(train_paths, train_labels, batch_size=32)
val_generator = generate_images(val_paths, val_labels, batch_size=32)

In [6]:
from tensorflow.keras.applications import EfficientNetV2B0
base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#  Fine-Tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # one output (tumor / healthy)

# bulid the model
model = Model(inputs=base_model.input, outputs=predictions)

# while training , trainable = false
for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_paths) // 32,
    validation_data=val_generator,
    validation_steps=len(val_paths) // 32,
    epochs=7
)

Epoch 1/7
221/221 ━━━━━━━━━━━━━━━━━━━━ 634s 3s/step - accuracy: 0.8997 - loss: 0.2344 - val_accuracy: 0.9767 - val_loss: 0.0826
Epoch 2/7
221/221 ━━━━━━━━━━━━━━━━━━━━ 576s 3s/step - accuracy: 0.9743 - loss: 0.0702 - val_accuracy: 0.9839 - val_loss: 0.0458
Epoch 3/7
221/221 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - accuracy: 0.9868 - loss: 0.0439 - val_accuracy: 0.9862 - val_loss: 0.0416
Epoch 4/7
221/221 ━━━━━━━━━━━━━━━━━━━━ 572s 3s/step - accuracy: 0.9871 - loss: 0.0373 - val_accuracy: 0.9850 - val_loss: 0.0428
Epoch 5/7
221/221 ━━━━━━━━━━━━━━━━━━━━ 618s 3s/step - accuracy: 0.9902 - loss: 0.0313 - val_accuracy: 0.9891 - val_loss: 0.0325
Epoch 6/7
221/221 ━━━━━━━━━━━━━━━━━━━━ 572s 3s/step - accuracy: 0.9921 - loss: 0.0260 - val_accuracy: 0.9891 - val_loss: 0.0322
Epoch 7/7
221/221 ━━━━━━━━━━━━━━━━━━━━ 568s 3s/step - accuracy: 0.9952 - loss: 0.0168 - val_accuracy: 0.9879 - val_loss: 0.0388


In [7]:
val_loss, val_accuracy = model.evaluate(val_generator, steps=len(val_paths) // 32)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

55/55 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.9870 - loss: 0.0439
Validation Loss: 0.03881007060408592
Validation Accuracy: 0.9879240989685059
